In [3]:
import tensorflow as tf
import tensorflow.compat.v2 as tfc
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util.tf_export import keras_export

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

import numpy as np
import sys
import pandas as pd
import sklearn as sk
import scipy as sp
import platform
import os


AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

My code starts here and I hope it ends today succesfully!!
Let's first import the dataset and split it into training 70%, validation 15% and testing 15%.

In [6]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
directory = "scrape"
train_dataset = image_dataset_from_directory(directory,
                                             labels='inferred',
                                             label_mode='categorical',
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)

valid_dataset = image_dataset_from_directory(directory,
                                             labels='inferred',
                                             label_mode='categorical',
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

NameError: name 'image_dataset_from_directory' is not defined

Because we don't have a training set we will take 20% of the validation set and turn it into a testing set.

In [44]:
val_batches = tf.data.experimental.cardinality(valid_dataset)
test_dataset = valid_dataset.take(val_batches // 5)
valid_dataset = valid_dataset.skip(val_batches // 5)

MAYBE NOT beacause it increases the size of the dataset, but we already have a lot of data so i don't wanna overcomplicate myself
let's augment the data to increase accuracy

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])

Then we load MobileNetV2:

In [45]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

Then we freeze the layers we don't want to train:

In [46]:
for layer in base_model.layers:
    layer.trainable = False

then we modify the last layers to fit to our goal


In [47]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

x = base_model.output
x = GlobalAveragePooling2D()(x)
#256 can be tuned and does not need to neccesarily match the classes no.
x = Dense(256, activation='relu')(x)
predictions = Dense(31, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

Compile the model

In [48]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train the model


In [49]:
model.fit(train_dataset, epochs=10, validation_data=valid_dataset)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: Shapes (None, 30) and (None, 31) are incompatible

Evaluate the model babyyyy


In [27]:
model.evaluate(test_dataset)

2/2 [==============================] - 2s 703ms/step - loss: 3.2520 - accuracy: 0.2344


[3.251995086669922, 0.234375]

Optional: Fine tune the model

In [28]:
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=10, validation_data=valid_dataset)


Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
56/56 [==============================] - ETA: 0s - loss: 3.0833 - accuracy: 0.1868WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000027C8937B550> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set 